In [383]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/movie-dataset/movies.csv
/kaggle/input/movie-dataset/ratings.csv
/kaggle/input/movie-dataset/tags.csv
/kaggle/input/movie-dataset/links.csv


In [384]:
import numpy as np
import csv
from collections import defaultdict
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
            
def data_load(mapping_movies):
#     print(mapping_movies)
    with open('/kaggle/input/movie-dataset/ratings.csv',newline='') as users:
        userRatings = csv.reader(users)
        next(userRatings)
        data = defaultdict(lambda:np.zeros(len(mapping_movies),dtype=float))
        for row in userRatings:
            data[row[0]][mapping_movies[row[1]]] = float(row[2])
        x=np.array([val for val in data.values()])
        return x,len(data)
        
    
n_users=0
n_movies=0
with open('/kaggle/input/movie-dataset/movies.csv',newline='') as movies:
    read = csv.reader(movies)
    next(read)
    count=0
    mapping_movies=defaultdict(lambda:0)
    for row in read:
        mapping_movies[row[0]] = count
        count+=1
    n_movies = len(mapping_movies)
    data,n_users=data_load(mapping_movies)

print("""We have organised the data in a matrix form where each row is indexed by the user Id
and contains his rating of each movie(if he has not rated it we have assumed it to be 0)      
""")




    


We have organised the data in a matrix form where each row is indexed by the user Id
and contains his rating of each movie(if he has not rated it we have assumed it to be 0)      



In [385]:
print("""We Now select a rating a user gives to a particular movie as Class Labels. 
To select this movie we can choose the most rated movie(with ratings > 0) 
""")

We Now select a rating a user gives to a particular movie as Class Labels. 
To select this movie we can choose the most rated movie(with ratings > 0) 



In [386]:
# for user in data:
id = np.count_nonzero(data > 0,axis=0).argmax()
print("The movie which is mapped to index '"+str(id)+"' is selected as the labeling movie")
print("Therefore we have labelled our data in Classse(ratings) of the selected movie")

The movie which is mapped to index '314' is selected as the labeling movie
Therefore we have labelled our data in Classse(ratings) of the selected movie


In [387]:
print("we now CLEAN DATA our data by removing the users who didnt rate the selected movie")
index=0
todelete=[]
for item in data:
    if item[id] == 0:
        todelete.append(index)
    index+=1
data=np.delete(data,todelete,axis=0)
print("Now the data has been reduced from 610 entries to 329 clean and contributing entries")

we now CLEAN DATA our data by removing the users who didnt rate the selected movie
Now the data has been reduced from 610 entries to 329 clean and contributing entries


In [388]:
print("""a record in data consists of "+str(n_movies-1)+" movie ratings(features) and rating 
of the selected movie as the label
""")
def classdistribution(data):
    classes=defaultdict(lambda:0)
    for label in data:
        classes[label[id]]+=1
    for rating,count in classes.items():
        print("Count of Class "+str(rating)+" : "+str(count))
    print()
classdistribution(data)
print("""To create a balanced dataset, we can use threshold and keep it at 4(>4 are liked and
less than 4 disliked)
""")

a record in data consists of "+str(n_movies-1)+" movie ratings(features) and rating 
of the selected movie as the label

Count of Class 4.0 : 94
Count of Class 5.0 : 116
Count of Class 3.0 : 39
Count of Class 3.5 : 27
Count of Class 4.5 : 39
Count of Class 2.0 : 9
Count of Class 1.0 : 1
Count of Class 0.5 : 1
Count of Class 2.5 : 3

To create a balanced dataset, we can use threshold and keep it at 4(>4 are liked and
less than 4 disliked)



In [389]:
print("We Now implement our custom Naive Bayes with 2 Classes(liked or disliked)")

lab = data[:,id]
temp = np.delete(data,id,1)
labels = []
ind=0
for record in temp:
    if lab[ind] > 4:
        labels.append('liked')
    else:
        labels.append('disliked')
    ind+=1
print(len(temp))

X_train, X_test, Y_train, Y_test = train_test_split(temp, labels, test_size=0.2,random_state=100)
model =  MultinomialNB(alpha=1.0, fit_prior=True)
model.fit(X_train,Y_train)
# prediction_prob = model.predict_proba(temp[-40:-1])
prediction = model.predict(X_test)
accuracy = model.score(X_test, Y_test)
print("the accuracy comes about: "+str(accuracy))
print(confusion_matrix(Y_test, prediction, labels=['liked', 'disliked']))
print("the accuracy is some bit low because of less data")

    


        
        
    
    

We Now implement our custom Naive Bayes with 2 Classes(liked or disliked)
329
the accuracy comes about: 0.5757575757575758
[[25 11]
 [17 13]]
the accuracy is low because of 
